In [ ]:
import pandas as pd
import numpy as np
import uuid
import joblib
from datetime import date

from lib.experience_data import pull_explanatory_variables, create_model
from lib.experience_data import process_data_for_analysis, get_latest_joblib
from lib.config import app, db
from lib.models import Hive, ModelHistory

Functions

Clean Hive Data for Study

In [2]:
with app.app_context():
    hives = [hive.to_dict() for hive in Hive.query.all()]

In [3]:
df_normalized, df_aggregated = process_data_for_analysis(hives)

df_aggregated.head()

,hive_id,date_added,city,state,honey_pull_id,date_reset,weight,date_pulled,count,temp,...,wax_moths_present,wasps_hornets_present,mice_present,robber_bees_present,has_chalkbrood,has_twisted_larvae,material_Polystyrene,material_Wood,days,avg_daily_weight
0,454,2023-03-03,Williamsville,Washington,1374.0,2023-03-03,20.936761,2023-08-16,23,21.626087,...,0,1,4,8,3,1,0,23,166,0.126125
1,454,2023-03-03,Williamsville,Washington,1375.0,2023-08-22,7.866216,2023-12-03,14,15.171429,...,1,2,3,5,0,0,0,14,103,0.076371
2,454,2023-03-03,Williamsville,Washington,1376.0,2023-12-15,8.866384,2024-04-07,16,5.0625,...,1,0,6,4,1,1,0,16,114,0.077775
3,454,2023-03-03,Williamsville,Washington,1377.0,2024-04-16,8.150840,2024-08-23,18,25.872222,...,2,0,0,6,0,2,0,18,129,0.063185
4,454,2023-03-03,Williamsville,Washington,1378.0,2024-09-04,9.734063,2024-12-23,15,12.653333,...,2,0,5,1,0,1,0,15,110,0.088491


Run Study and Save to Joblib

In [4]:
explanatory_variables=pull_explanatory_variables(df_aggregated)
joblib_loc = f'exp_study{uuid.uuid4().hex}.joblib'

model, test_results, importance_df = create_model(df_aggregated, explanatory_variables, joblib_loc)

results = test_results[['weight', 'avg_daily_weight', 'days', 'avg_predicted', 'predicted']].copy()

results

,weight,avg_daily_weight,days,avg_predicted,predicted
9,18.330655,0.132831,138,0.145465,20.074199
11,16.103344,0.095286,169,0.132456,22.385072
0,20.936761,0.126125,166,0.117848,19.562818
12,4.883235,0.045638,107,0.151124,16.170256
5,15.621111,0.138240,113,0.135994,15.367271


Save Joblib location to Database

In [5]:
with app.app_context():
    # Save metadata for new model to database
    new_study = ModelHistory(
        joblib_loc = joblib_loc
    )

    # Add the new study to the database and commit
    db.session.add(new_study)
    db.session.commit()

Update active model in Database

In [6]:
with app.app_context():
    prior_model = ModelHistory.query.filter(
        ModelHistory.end_date == None, 
        ModelHistory.start_date != None
    ).first()

    # Set the end date
    prior_model.end_date = date.today()

    current_model = ModelHistory.query.filter_by(
        joblib_loc=joblib_loc
    ).first()

    # Set the start and end dates
    current_model.start_date = date.today()
    current_model.end_date = None

    # Commit changes to the database
    db.session.commit()

In [7]:
joblib_loc = exps.get_latest_joblib()
df_normalized, df_prediction_input = dclean.process_data_for_analysis(hives, actuals=False)
predicted_values = exps.run_predictions(df_prediction_input, joblib_loc)
predictions_only = predicted_values[['hive_id', 'predicted']].set_index('hive_id')

prediction_dict = predictions_only.to_dict()
prediction_dict

NameError: name 'exps' is not defined

Testing

In [ ]:
hives_mod = dclean.rename_ids(hives)
print('renamed ids...')

df_normalized = dclean.normalize_data(hives_mod)
print('df normalized...')

df_aggregated = dclean.aggregate_data(df_normalized)
print('df aggregated...')

json_normalized = df_normalized.to_dict(orient='list')
print('json normalized...')

json_aggregated = df_aggregated.to_dict(orient='list')
print('json aggregated...')

renamed ids...
df normalized...
df aggregated...
json normalized...
json aggregated...
